In [ ]:
import pandas as pd
import numpy as np
from aif360.datasets import AdultDataset
from aif360.algorithms.preprocessing import Reweighing
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from aif360.metrics import ClassificationMetric

# Load the dataset
dataset = AdultDataset(protected_attribute_names=['sex'],
                       privileged_classes=[['Male']],
                       features_to_drop=['education-num'])

# Split into train and test
dataset_train, dataset_test = dataset.split([0.7], shuffle=True)

# Apply reweighing
RW = Reweighing(unprivileged_groups=[{'sex': 0}],  # 0 = Female
                privileged_groups=[{'sex': 1}])    # 1 = Male
RW.fit(dataset_train)
dataset_train_transf = RW.transform(dataset_train)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(dataset_train_transf.features)
y_train = dataset_train_transf.labels.ravel()
sample_weight = dataset_train_transf.instance_weights

X_test = scaler.transform(dataset_test.features)
y_test = dataset_test.labels.ravel()

# Train logistic regression model
clf = LogisticRegression(solver='liblinear')
clf.fit(X_train, y_train, sample_weight=sample_weight)

# Predict on test data
y_pred = clf.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Fairness Metrics
pred_dataset = dataset_test.copy()
pred_dataset.labels = y_pred.reshape(-1, 1)

metric = ClassificationMetric(dataset_test, pred_dataset,
                               unprivileged_groups=[{'sex': 0}],
                               privileged_groups=[{'sex': 1}])

print("Disparate Impact:", metric.disparate_impact())
print("Statistical Parity Difference:", metric.statistical_parity_difference())